In [362]:
!git commit -am "WIP refactored code"
!git push

[binder 7b3221e] WIP refactored code
 1 file changed, 259 insertions(+), 83 deletions(-)
Counting objects: 4, done.
Delta compression using up to 32 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.37 KiB | 1.37 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/yoyu777/jupyter_notebooks.git
   6dc0f85..7b3221e  binder -> binder


In [262]:
!pip install  lxml pandas
from urllib.request import urlopen
from lxml import html, etree
import pandas as pd
import numpy as np
import json

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [114]:
def extractText(ele):
    return ele.text.strip()

In [346]:
def extractHoldingsFromStock(url):
    print(url)
    page=urlopen(url)
    tree = html.fromstring(page.read().decode('utf-8'))
    holding=tree.xpath('//div[@class="holding"]')[0]
    tbody=holding.xpath('//tbody')[0]
    holding={
        'FullName':tbody.xpath('//div[@data-type="full"]'),
        'Abbreviation':tbody.xpath('//div[@data-type="abbreviation"]'),
        'Percentage':tbody.xpath('//td[@data-type="percent"]')
    }
    df = pd.DataFrame(data=holding)
    df=df.apply(lambda s:s.apply(extractText))
    return df

In [347]:
funds=[]
funds.append({
        'Name':'Janus Henderson Global Technology Fund',
        'URL':'https://www.bloomberg.com/quote/JAGTX:US',
        'Allocation':4
    })
funds.append({
        'Name':'iShares Healthcare Innovation UCITS ETF',
        'URL':'https://www.bloomberg.com/quote/HEAL:SW',
        'Allocation':2
    })

In [348]:
df=pd.DataFrame(funds,columns=['Name','URL','Allocation'])
df

,Name,URL,Allocation
0,Janus Henderson Global Technology Fund,https://www.bloomberg.com/quote/JAGTX:US,4
1,iShares Healthcare Innovation UCITS ETF,https://www.bloomberg.com/quote/HEAL:SW,2


In [349]:
df['Holdings']=df.URL.apply(extractHoldingsFromStock)

https://www.bloomberg.com/quote/JAGTX:US
https://www.bloomberg.com/quote/HEAL:SW


In [350]:
sum=df.Allocation.sum()
weights=list(map(lambda v:round(v/sum,2),df.Allocation))
df['OverallWeight']=weights
df[['Name','OverallWeight']]

,Name,OverallWeight
0,Janus Henderson Global Technology Fund,0.67
1,iShares Healthcare Innovation UCITS ETF,0.33


In [351]:
df

,Name,URL,Allocation,Holdings,OverallWeight
0,Janus Henderson Global Technology Fund,https://www.bloomberg.com/quote/JAGTX:US,4,FullName Abbreviation...,0.67
1,iShares Healthcare Innovation UCITS ETF,https://www.bloomberg.com/quote/HEAL:SW,2,FullName Abbreviation...,0.33


In [358]:
def factorInWeights(row):
#     print(row.Name)
    row.Holdings['OverallPercentage']=row.Holdings.Percentage.apply(lambda v:float(v)*row.OverallWeight)
    return 

df.apply(factorInWeights,axis=1)

0    None
1    None
dtype: object

In [359]:
def appendFundName(row):
    row.Holdings['FundName']=row.Name
    return
df.apply(appendFundName,axis=1)

0    None
1    None
dtype: object

In [ ]:
allHoldings=pd.concat(df.Holdings.values)
allHoldings
allHoldings.groupby('Abbreviation').sort_values(['OverallPercentage', 'Percentage'], ascending=[0,0])